In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
!git clone https://github.com/open-mmlab/mmsegmentation.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 46.4 MB 156 kB/s 
     |████████████████████████████████| 190 kB 8.1 MB/s 
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 7196, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 7196 (delta 54), reused 157 (delta 43), pack-reused 6995
Receiving objects: 100% (7196/7196), 13.43 MiB | 16.47 MiB/s, done.
Resolving deltas: 100% (5247/5247), done.


In [3]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu113 (Tesla P100-PCIE-16GB)


In [4]:
%cd mmsegmentation

/content/mmsegmentation


In [5]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
import mmcv

In [6]:
import os.path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
!mkdir checkpoints
!wget -O //content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth

--2022-06-02 02:28:46--  https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.72
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328290585 (313M) [application/octet-stream]
Saving to: ‘//content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth’

//content/mmsegment 100%[===================>] 313.08M  13.4MB/s    in 23s     

2022-06-02 02:29:10 (13.4 MB/s) - ‘//content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth’ saved [328290585/328290585]



In [8]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

classes = ('background', 'road')
palette = [[0, 0, 0], [255, 255, 0]]

@DATASETS.register_module()
class SIADataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.png', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

In [9]:
# config 파일을 설정하고, 다운로드 받은 pretrained 모델을 checkpoint로 설정. 
config_file = '/content/mmsegmentation/configs/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py'
checkpoint_file = '/content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1,
        init_cfg=dict(type='Pretrained', checkpoint='pretrain/mit_b5.pth')),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=19,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    train_cf

In [10]:
model = dict(
    type='EncoderDecoder',
    pretrained='open-mmlab://resnet101_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 1, 1),
        strides=(1, 2, 2, 2),
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=4),
    decode_head=dict(
        type='FPNHead',
        in_channels=[256, 256, 256, 256],
        in_index=[0, 1, 2, 3],
        feature_strides=[4, 8, 16, 32],
        channels=128,
        dropout_ratio=0.1,
        num_classes=19,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    train_cfg=dict(),
    test_cfg=dict(mode='whole'))

In [11]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
crop_size = (512, 512)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(1024, 1024), ratio_range=(0.5, 1.5)),
    dict(type='RandomCrop', crop_size=(512, 512), cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size=(512, 512), pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [12]:
dataset_type = 'CityscapesDataset'
data_root = 'data/cityscapes/'

data = dict(
    samples_per_gpu=2,  #batch size
    workers_per_gpu=2,  
    train=dict(
        type='CityscapesDataset',
        data_root='data/cityscapes/',
        img_dir='leftImg8bit/train',
        ann_dir='gtFine/train',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='LoadAnnotations'),
            dict(
                type='Resize', img_scale=(1024, 1024), ratio_range=(0.5, 1.5)),
            dict(type='RandomCrop', crop_size=(512, 512), cat_max_ratio=0.75),
            dict(type='RandomFlip', prob=0.5),
            dict(type='PhotoMetricDistortion'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size=(512, 512), pad_val=0, seg_pad_val=255),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img', 'gt_semantic_seg'])
        ]),
    val=dict(
        type='CityscapesDataset',
        data_root='data/cityscapes/',
        img_dir='leftImg8bit/val',
        ann_dir='gtFine/val',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1024, 1024),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ]),
    test=dict(
        type='CityscapesDataset',
        data_root='data/cityscapes/',
        img_dir='leftImg8bit/val',
        ann_dir='gtFine/val',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1024, 1024),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ]))

In [13]:
cfg.norm_cfg = dict(type='BN', requires_grad=True)
#cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg

cfg.model.decode_head.num_classes = 2

cfg.device='cuda'

cfg.model.decode_head.sampler=dict(type='OHEMPixelSampler', thresh=0.7, min_kept=100000)

cfg.model.decode_head.loss_decode = [dict(type='DiceLoss', loss_weight = 1.0),                                    
                                     dict(type='LovaszLoss', loss_weight = 1.0, reduction='none')]

In [14]:
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (512, 512)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(1024, 1024), ratio_range=(0.5, 1.5)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.val_pipeline = [
                    dict(type='LoadImageFromFile'),
                    dict(
                        type='MultiScaleFlipAug',
                        img_scale=(1024, 1024),
                        flip=False,
                        transforms=[
                                    dict(type='Resize', keep_ratio=True),
                                    dict(type='RandomFlip'),
                                    dict(
                                        type='Normalize',
                                        mean=[123.675, 116.28, 103.53],
                                        std=[58.395, 57.12, 57.375],
                                        to_rgb=True),
                                    dict(type='ImageToTensor', keys=['img']),
                                    dict(type='Collect', keys=['img'])
                                    ]),
                    
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            #dict(type='ResizeToMultiple', size_divisor=32),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

In [15]:
cfg.dataset_type = 'SIADataset'
cfg.data_root = '/content/drive/MyDrive/SIA'

cfg.data.train.type = 'SIADataset'
cfg.data.train.data_root = '/content/drive/MyDrive/SIA'
cfg.data.train.img_dir = 'Data_set/road_dataset/img_dir/train'
cfg.data.train.ann_dir = 'Data_set/road_dataset/ann_01_dir/train'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = '/content/drive/MyDrive/SIA/Data_set/road_dataset/mask_dir/train/up_train_road.txt'

cfg.data.val.type = 'SIADataset'
cfg.data.val.data_root = '/content/drive/MyDrive/SIA'
cfg.data.val.img_dir = 'Data_set/road_dataset/img_dir/train'
cfg.data.val.ann_dir = 'Data_set/road_dataset/ann_01_dir/train'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = '/content/drive/MyDrive/SIA/Data_set/road_dataset/mask_dir/train/up_val_road.txt'

cfg.data.test.type = 'SIADataset'
cfg.data.test.data_root = '/content/drive/MyDrive/SIA'
cfg.data.test.img_dir = 'Data_set/road_dataset/img_dir/val'
cfg.data.test.ann_dir = 'Data_set/road_dataset/ann_01_dir/val'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = '/content/drive/MyDrive/SIA/Data_set/road_dataset/mask_dir/val/up.txt'

In [16]:
'''
cfg.data.train.type = 'SIADataset'
cfg.data.train.data_root = '/content/drive/MyDrive/SIA'
cfg.data.train.img_dir = 'images/up025'
cfg.data.train.ann_dir = 'labels'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = '/content/drive/MyDrive/SIA/splits/road/train_road.txt'
'''

"\ncfg.data.train.type = 'SIADataset'\ncfg.data.train.data_root = '/content/drive/MyDrive/SIA'\ncfg.data.train.img_dir = 'images/up025'\ncfg.data.train.ann_dir = 'labels'\ncfg.data.train.pipeline = cfg.train_pipeline\ncfg.data.train.split = '/content/drive/MyDrive/SIA/splits/road/train_road.txt'\n"

In [17]:
cfg.load_from = '/content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DL_sampler'

In [ ]:
cfg.runner.max_iteTrs = 200
cfg.log_config.interval = 100
cfg.evaluation.interval = 1000  # 모델 학습시 평가를 몇 번째 iteration마다 할 것인지 지정
cfg.checkpoint_config.interval = 1000  # 모델 학습시 학습한 모델을 몇 번째 iteration마다 저장할 것인지 지정

cfg.runner = dict(type='IterBasedRunner', max_iters=20000)  # Iteration으로 동작, Epoch로 동작하게 변경할 수도 있음
# cfg.runner = dict(type='EpochBasedRunner', max_epochs=4000)  # Epoch로 변경
cfg.workflow = [('train', 1)]

# Set seed to facitate reproducing the result
cfg.seed = 0
#set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1,
        init_cfg=dict(type='Pretrained', checkpoint='pretrain/mit_b5.pth')),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=2,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=False,
        loss_decode=[
            dict(type='DiceLoss', loss_weight=1.0),
            dict(type='LovaszLoss', loss

### 모델 학습

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True,
                meta=dict(CLASSES=classes, PALETTE=palette))

2022-05-27 00:52:04,019 - mmseg - INFO - Loaded 542 images
2022-05-27 00:52:19,462 - mmseg - INFO - Loaded 66 images
2022-05-27 00:52:19,465 - mmseg - INFO - load checkpoint from local path: /content/mmsegmentation/checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth
2022-05-27 00:52:19,898 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([19, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 256, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([2]).
2022-05-27 00:52:19,904 - mmseg - INFO - Start running, host: root@95063dcd7a87, work_dir: /content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DL_sampler
2022-05-27 00:52:19,905 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.1 task/s, elapsed: 482s, ETA:     0s

2022-05-27 01:17:57,582 - mmseg - INFO - per class results:
2022-05-27 01:17:57,585 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 89.33 | 93.77 |
|    road    |  58.2 | 75.81 |
+------------+-------+-------+
2022-05-27 01:17:57,586 - mmseg - INFO - Summary:
2022-05-27 01:17:57,589 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 90.71 | 73.76 | 84.79 |
+-------+-------+-------+
2022-05-27 01:17:57,593 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9071, mIoU: 0.7376, mAcc: 0.8479, IoU.background: 0.8933, IoU.road: 0.5820, Acc.background: 0.9377, Acc.road: 0.7581
2022-05-27 01:18:39,754 - mmseg - INFO - Iter [1100/20000]	lr: 4.154e-05, eta: 7:32:23, time: 5.243, data_time: 4.847, memory: 3323, decode.loss_dice: 0.1544, decode.loss_lovasz: 0.2941, decode.acc_seg: 89.4384, loss: 0.4485
2022-05-27 01:19:19,884 - mmseg - INFO - Iter [1200/20000]	lr: 4.508e-05, eta: 7:02:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.2 task/s, elapsed: 436s, ETA:     0s

2022-05-27 01:32:06,527 - mmseg - INFO - per class results:
2022-05-27 01:32:06,529 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 89.91 | 94.18 |
|    road    | 59.94 | 76.88 |
+------------+-------+-------+
2022-05-27 01:32:06,533 - mmseg - INFO - Summary:
2022-05-27 01:32:06,535 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 91.23 | 74.92 | 85.53 |
+-------+-------+-------+
2022-05-27 01:32:06,538 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9123, mIoU: 0.7492, mAcc: 0.8553, IoU.background: 0.8991, IoU.road: 0.5994, Acc.background: 0.9418, Acc.road: 0.7688
2022-05-27 01:32:47,065 - mmseg - INFO - Iter [2100/20000]	lr: 5.370e-05, eta: 5:44:47, time: 4.761, data_time: 4.362, memory: 3323, decode.loss_dice: 0.1494, decode.loss_lovasz: 0.2773, decode.acc_seg: 89.8368, loss: 0.4268
2022-05-27 01:33:29,238 - mmseg - INFO - Iter [2200/20000]	lr: 5.340e-05, eta: 5:32:

...

2022-05-27 05:19:05,299 - mmseg - INFO - per class results:
2022-05-27 05:19:05,302 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 92.06 | 96.16 |
|    road    | 66.07 | 78.41 |
+------------+-------+-------+
2022-05-27 05:19:05,303 - mmseg - INFO - Summary:
2022-05-27 05:19:05,304 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 93.13 | 79.07 | 87.28 |
+-------+-------+-------+
2022-05-27 05:19:05,306 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9313, mIoU: 0.7907, mAcc: 0.8728, IoU.background: 0.9206, IoU.road: 0.6607, Acc.background: 0.9616, Acc.road: 0.7841
2022-05-27 05:19:45,397 - mmseg - INFO - Iter [18100/20000]	lr: 5.703e-06, eta: 0:28:04, time: 4.848, data_time: 4.452, memory: 3323, decode.loss_dice: 0.0917, decode.loss_lovasz: 0.1861, decode.acc_seg: 93.6420, loss: 0.2778
2022-05-27 05:20:25,456 - mmseg - INFO - Iter [18200/20000]	lr: 5.403e-06, eta: 0:2

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.1 task/s, elapsed: 445s, ETA:     0s

2022-05-27 05:33:20,332 - mmseg - INFO - per class results:
2022-05-27 05:33:20,335 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 92.03 | 96.15 |
|    road    | 65.93 | 78.27 |
+------------+-------+-------+
2022-05-27 05:33:20,337 - mmseg - INFO - Summary:
2022-05-27 05:33:20,339 - mmseg - INFO - 
+------+-------+-------+
| aAcc |  mIoU |  mAcc |
+------+-------+-------+
| 93.1 | 78.98 | 87.21 |
+------+-------+-------+
2022-05-27 05:33:20,343 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9310, mIoU: 0.7898, mAcc: 0.8721, IoU.background: 0.9203, IoU.road: 0.6593, Acc.background: 0.9615, Acc.road: 0.7827
2022-05-27 05:34:00,330 - mmseg - INFO - Iter [19100/20000]	lr: 2.703e-06, eta: 0:13:16, time: 4.846, data_time: 4.451, memory: 3323, decode.loss_dice: 0.0848, decode.loss_lovasz: 0.1757, decode.acc_seg: 94.0366, loss: 0.2605
2022-05-27 05:34:40,408 - mmseg - INFO - Iter [19200/20000]	lr: 2.403e-06, eta: 0:11:45,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 66/66, 0.1 task/s, elapsed: 444s, ETA:     0s

2022-05-27 05:47:34,867 - mmseg - INFO - per class results:
2022-05-27 05:47:34,869 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 92.12 | 96.32 |
|    road    | 66.03 | 77.83 |
+------------+-------+-------+
2022-05-27 05:47:34,872 - mmseg - INFO - Summary:
2022-05-27 05:47:34,875 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 93.17 | 79.08 | 87.08 |
+-------+-------+-------+
2022-05-27 05:47:34,877 - mmseg - INFO - Iter(val) [66]	aAcc: 0.9317, mIoU: 0.7908, mAcc: 0.8708, IoU.background: 0.9212, IoU.road: 0.6603, Acc.background: 0.9632, Acc.road: 0.7783


### 모델 결과 시각화

In [ ]:
img_result = ['BLD00002_PS3_K3A_NIA0276.png',
              'BLD00010_PS3_K3A_NIA0276.png',
              'BLD00047_PS3_K3A_NIA0276.png',
              'BLD12071_PS3_K3A_NIA0391.png',
              'BLD11907_PS3_K3A_NIA0391.png',
              'BLD11611_PS3_K3A_NIA0390.png',
              'BLD11474_PS3_K3A_NIA0390.png',
              'BLD10413_PS3_K3A_NIA0388.png',
              'BLD10150_PS3_K3A_NIA0388.png',
              'BLD00836_PS3_K3A_NIA0277.png']

checkpoint_file = '/content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DL_sampler/iter_20000.pth'  #학습된 모델


# checkpoint 저장된 model 파일을 이용하여 모델을 생성, 이때 Config는 위에서 update된 config 사용. 
model_ckpt = init_segmentor(cfg, checkpoint_file, device='cuda:0')  #cuda gpu 사용

for ir in range(len(img_result)):
  img = mmcv.imread('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val/'+img_result[ir])
  result = inference_segmentor(model_ckpt, img)
  show_result_pyplot(model_ckpt, img, result, palette)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import os
file_list = os.listdir('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val')
test_png = []
for i in file_list:
  if '.png' in i:
    test_png.append(i)

for j in test_png:
  img = mmcv.imread('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val/'+j)
  result = inference_segmentor(model_ckpt, img)
  show_result_pyplot(model_ckpt, img, result, palette)

Output hidden; open in https://colab.research.google.com to view.

In [18]:
import os
file_list = os.listdir('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val')
test_png = []
for i in file_list:
  if '.png' in i:
    test_png.append(i)

checkpoint_file = '/content/drive/MyDrive/SIA/semantic_checkpoint/Segformer/road/DL_sampler/iter_20000.pth'  #학습된 모델  #change point
model_ckpt = init_segmentor(cfg, checkpoint_file, device='cuda:0')  #cuda gpu 사용
img2 = np.zeros((1024,1024,3),np.uint8)

for j in test_png:
  img = mmcv.imread('/content/drive/MyDrive/SIA/Data_set/road_dataset/img_dir/val/'+j)
  result = inference_segmentor(model_ckpt, img)
  show_result_pyplot(model_ckpt, img2, result, palette)
  model_ckpt.show_result(img2, result, out_file= '/content/drive/MyDrive/SIA/Morphological/DL/sampler/Dataset/'+j) #change point

Output hidden; open in https://colab.research.google.com to view.